In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /home/pranshu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pranshu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
questions = {'type1':[] , 'type2':[] , 'type3':[]}

In [25]:
data_file = open('intents1.json').read()
intents = json.loads(data_file)
for i in intents['type1']:
    for question in i['patterns']:
        if question != '':
            questions['type1'].append(question)
        
data_file = open('intents2.json').read()
intents = json.loads(data_file)
for i in intents['type2']:
    for question in i['patterns']:
        if question != '':
            questions['type2'].append(question)
        
data_file = open('intents3.json').read()
intents = json.loads(data_file)
for i in intents['type3']:
    for question in i['patterns']:
        if question != '':
            questions['type3'].append(question)

In [26]:
print(questions)

{'type1': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day', 'Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time', 'Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me', 'How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered', 'Who made you', 'Who created you', 'Who is your maker', 'Who is your creator', 'Who built you', 'Who constructed you'], 'type2': ['How is the campus of IIT Mand', 'What is the branch change requirement', 'What is the branch change criteria', 'What are the branch change conditions', 'What is the branch switch criteria', 'What does it take to change the branch', 'Is it easy to change the branch', 'When is the changed branch allocated', 'What is the branch change policy at IIT Mandi', 'What is the minimum cgpa required for branch change', 'What is the minimum CG required for branch change', 'Minimum pointer f

In [27]:
words=[]
classes = ['type1' , 'type2' , 'type3']
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [29]:
for qtype in questions:
    for question in questions[qtype]:
        w = nltk.word_tokenize(question)
        words.extend(w)
        documents.append((w, qtype))

In [30]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents" , documents)

# print (len(classes), "classes", classes)

# print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

143 documents [(['Hi', 'there'], 'type1'), (['How', 'are', 'you'], 'type1'), (['Is', 'anyone', 'there', '?'], 'type1'), (['Hey'], 'type1'), (['Hola'], 'type1'), (['Hello'], 'type1'), (['Good', 'day'], 'type1'), (['Bye'], 'type1'), (['See', 'you', 'later'], 'type1'), (['Goodbye'], 'type1'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'type1'), (['Till', 'next', 'time'], 'type1'), (['Thanks'], 'type1'), (['Thank', 'you'], 'type1'), (['That', "'s", 'helpful'], 'type1'), (['Awesome', ',', 'thanks'], 'type1'), (['Thanks', 'for', 'helping', 'me'], 'type1'), (['How', 'you', 'could', 'help', 'me', '?'], 'type1'), (['What', 'you', 'can', 'do', '?'], 'type1'), (['What', 'help', 'you', 'provide', '?'], 'type1'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'type1'), (['What', 'support', 'is', 'offered'], 'type1'), (['Who', 'made', 'you'], 'type1'), (['Who', 'created', 'you'], 'type1'), (['Who', 'is', 'your', 'maker'], 'type1'), (['Who', 'is', 'your', 'creator'], 'type1'), (['Who', 'built', '

In [15]:
# Classes has list of intents
# Documents : (['words','in', 'the' , 'message'] , 'intent')
# Words : All words in sorted order

In [31]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # print(pattern_words)
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # print(bag)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-31-76e24bbee991>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [32]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")

Epoch 1/200
29/29 [==============================] - 0s 1ms/step - loss: 1.0498 - accuracy: 0.4545
Epoch 2/200
29/29 [==============================] - 0s 1ms/step - loss: 0.8743 - accuracy: 0.6294
Epoch 3/200
29/29 [==============================] - 0s 1ms/step - loss: 0.7586 - accuracy: 0.6643
Epoch 4/200
29/29 [==============================] - 0s 986us/step - loss: 0.6459 - accuracy: 0.7063
Epoch 5/200
29/29 [==============================] - 0s 994us/step - loss: 0.5046 - accuracy: 0.7622
Epoch 6/200
29/29 [==============================] - 0s 1ms/step - loss: 0.4426 - accuracy: 0.8322
Epoch 7/200
29/29 [==============================] - 0s 903us/step - loss: 0.3329 - accuracy: 0.8741
Epoch 8/200
29/29 [==============================] - 0s 947us/step - loss: 0.2581 - accuracy: 0.9231
Epoch 9/200
29/29 [==============================] - 0s 955us/step - loss: 0.2256 - accuracy: 0.9371
Epoch 10/200
29/29 [==============================] - 0s 969us/step - loss: 0.1155 - accuracy: 0.97

29/29 [==============================] - 0s 1ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 83/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 84/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0113 - accuracy: 0.9930
Epoch 85/200
29/29 [==============================] - 0s 993us/step - loss: 0.0087 - accuracy: 1.0000
Epoch 86/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 87/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 88/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 89/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 90/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 91/200
29/29 [==============================] - 0s 1ms/step - loss: 8.5052e-04 - accuracy: 1.0000
Epoch 9

Epoch 162/200
29/29 [==============================] - 0s 952us/step - loss: 9.8128e-04 - accuracy: 1.0000
Epoch 163/200
29/29 [==============================] - 0s 906us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 164/200
29/29 [==============================] - 0s 1ms/step - loss: 3.9187e-04 - accuracy: 1.0000
Epoch 165/200
29/29 [==============================] - 0s 1ms/step - loss: 4.2023e-04 - accuracy: 1.0000
Epoch 166/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0076 - accuracy: 1.0000
Epoch 167/200
29/29 [==============================] - 0s 877us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 168/200
29/29 [==============================] - 0s 858us/step - loss: 6.7731e-04 - accuracy: 1.0000
Epoch 169/200
29/29 [==============================] - 0s 840us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 170/200
29/29 [==============================] - 0s 917us/step - loss: 0.0022 - accuracy: 1.0000
Epoch 171/200
29/29 [==============================] - 0s 951us

In [34]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('model.h5')
import json
import random
intents = json.loads(open('intents1.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [35]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [36]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [37]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    print(return_list)
    return return_list

In [38]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [46]:
def chatbot_response(msg):
    ints = predict_class(msg, model)

In [62]:
chatbot_response("deepali meet pe aagyi guys")

[{'intent': 'type1', 'probability': '0.97331333'}]
